In [ ]:
# reload notebook automatically after changes to source python files
%load_ext autoreload
%autoreload 2

# change base folder to parent
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(os.getcwd())

import torch
torch.hub.set_dir('/scratch/toponav/indoor-topo-loc/checkpoints/torchhub')
print(torch.hub.get_dir())  

In [ ]:
from pathlib import Path
from configs.default import cfg
from src.utils.viz_utils import load_image, plot_images

eval_spp_config = 'configs/config_eval_spp_resz.yaml'
cfg.merge_from_file(eval_spp_config)

CURR_SCENE='e8e81396b6'
# FRAME0_NAME, FRAME1_NAME = 'DSC05283', 'DSC05285' # 0-45
# FRAME0_NAME, FRAME1_NAME = 'DSC05283', 'DSC05181' # 45-90
# FRAME0_NAME, FRAME1_NAME = 'DSC05283', 'DSC05222' # 90-135
FRAME0_NAME, FRAME1_NAME = 'DSC05283', 'DSC05214' # 135-180

dslr_path = Path(cfg["DATASET"]["DATA_ROOT"])
segdata_orig_path = Path(cfg["DATASET"]["SEGDATA_ROOT"])

# load frames
frame0_path = dslr_path / CURR_SCENE / f"{FRAME0_NAME}.JPG"
frame1_path = dslr_path / CURR_SCENE / f"{FRAME1_NAME}.JPG"

frame0 = load_image(frame0_path)
frame1 = load_image(frame1_path)

plot_images(frame0, frame1)


frame0_segdata_path = segdata_orig_path / CURR_SCENE / f"{FRAME0_NAME}.JPG.pkl"
frame1_segdata_path = segdata_orig_path / CURR_SCENE / f"{FRAME1_NAME}.JPG.pkl"

In [ ]:
import torch
from PIL import Image
import torch.nn.functional as F
import numpy as np
from romatch.utils.utils import tensor_to_pil

from romatch import roma_outdoor, tiny_roma_v1_outdoor, roma_indoor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

im1_path = str(frame0_path)
im2_path = str(frame1_path)
save_path = "plot_roma_indoor_0_45.png"

# Create model
roma_model = roma_indoor(device=device, coarse_res=560, upsample_res=(864, 1152))
roma_model.symmetric = False

H, W = roma_model.get_output_resolution()

# Match
warp, certainty = roma_model.match(im1_path, im2_path, device=device)
# Sample matches for estimation
matches, certainty = roma_model.sample(warp, certainty)
# Convert to pixel coordinates (RoMa produces matches in [-1,1]x[-1,1])
kptsA, kptsB = roma_model.to_pixel_coordinates(matches, 336, 512, 336, 512)